- 所有文件都会先经过ocr识别处理
- ocr后的结果应该是一个大的json文件，该文件应：
    - 嵌套式体现文档结构，表明哪些是标题及其标题等级
    - 文档内容应区分表格内容（有开头结尾标记）和其他
    - 内容跨页需要合并
- 由于ocr接正在开发中，本方案依据理想化的ocr结果编写

In [37]:
from paddlenlp import Taskflow
import pandas as pd
import json
from IPython.display import display
import re
import os
from translate import Translator

# Helping Functions

## extract_info_from_taskflow

In [2]:
def extract_info_from_taskflow(task, case, schema):
    '''
    task: taskflow with pre-defined sechema
    case: string, info for analysing
    schema: key-words needs to be extracted
    '''
    info = task(case)  ## paddleNLP taskflow 模型抽取信息
    result = {}  ## 存放抽取结果
    for k in schema:
        v = info[0].get(k)
        if v:
            v = v[0].get("text")
        result[k] = v
    return result

## key translation

In [92]:
trans_dict_path = "./data/key2en.json"
translator = Translator(from_lang="ZH",to_lang="EN")

def key_2_English(key, translator=translator, 
                  trans_dict_path=trans_dict_path, 
                  force_translate=False, update_trans_dict_file=True):
    '''
    用translate包将key转换成英文
    并维护一个中转英的dict
    可以人为修改对应英文

    key: 一个要翻译的字符串
    translator: Translator(from_lang="ZH",to_lang="EN")
    trans_dict_path: 存储中英字符对应翻译

    force_translate: 强制重新翻译，也会更新表格
    update_trans_dict_file：是否更新存储的dict
    '''
    ## 检查是否含有中文
    contains_ZH = False
    for char in key:
        if '\u4e00' <= char <= '\u9fff':
            contains_ZH = True
            break
    if not contains_ZH:
        return key
        
    need_updating = False  ## flag for 是否需要更新字典

    ## 载入已有dict, 或创立新dict
    if not os.path.exists(trans_dict_path):
        need_updating = True
        trans_dict = {}
    else:
        with open(trans_dict_path, 'r') as file:
            trans_dict = json.load(file)

    key_en = trans_dict.get(key)
    if force_translate:
        key_en = None
    if not key_en:
        need_updating = True
        key_en = translator.translate(key)

        matches = re.findall(">(.*?)<", key_en)  ## 有时会有<bold>..<>修饰
        if matches:
            key_en = matches[0].strip(":")

        ## 原有下划线为标题分级
        key_en = re.sub(r'\_', '\_\_', key_en)  
        ## 转小写
        ## 删除非英文字母非空格的字符
        key_en = re.sub(r'[^a-zA-Z0-9\_\s]', '', key_en.lower())  
        ## 下划线连接
        key_en = '_'.join(key_en.split())
        
        ## 更新dict
        trans_dict[key] = key_en
        if update_trans_dict_file:
            with open(trans_dict_path, 'w') as file:
                json.dump(trans_dict, file)

    return key_en

def update_trans_dict(key, key_en, trans_dict_path=trans_dict_path):
    '''
    手动修改英文翻译
    '''
    need_updating = False

    ## 载入已有dict, 或创立新dict
    if not os.path.exists(trans_dict_path):
        need_updating = True
        trans_dict = {}
    else:
        with open(trans_dict_path, 'r') as file:
            trans_dict = json.load(file)

    ## 更新dict
    trans_dict[key] = key_en
    with open(trans_dict_path, 'w') as file:
        json.dump(trans_dict, file)

def key_list_2_English(keys):
    return [key_2_English(k) for k in keys]

## split cases

In [15]:
def split_cases_by_idx_from_paragraph(data, split_pattern=r'\s+\d+\.\s*'): 
    '''
    用于分段：
        1. 。。。
        2. 。。。
    '''
    data = '\n' + data.strip().strip("\n")

    ## 分段标准: 一个或多个空白 + 一个或多个数字 + . + 空格
    data_splited = re.split(split_pattern, data)

    cases = []
    for case in data_splited:
        if case == '':
            continue
        ## 去头尾空格，去所有\n
        case = case.strip().replace("\n","")
        cases.append(case)
    return cases

# 个人信用报告

## == 内容样例

![个人信用报告](4_简版方案/个人信用报告.png)

In [54]:
grxybg_data = """
\n报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15\n姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚\n其他证件信息：护照G300234234/军人身份证 M09876893\n
"""
grxybg_data

'\n\n报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15\n姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚\n其他证件信息：护照G300234234/军人身份证 M09876893\n\n'

In [55]:
def grxybg_text_preprocess(data):
    data = data.strip().strip("\n").strip()
    data = data.replace("\n", "  ")
    return data

grxybg_data = grxybg_text_preprocess(data=grxybg_data)
grxybg_data

'报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15  姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚  其他证件信息：护照G300234234/军人身份证 M09876893'

## == 预期提取效果

整体效果（提取结果存放在字段 `info_extraction`）：
```json
{
    'header-0':{
        'header-name':'个人信用报告',
        'content':[ocr返回结果],
        'info_extraction':[result]
    }
}
```
`result` 关键字段及其中英对照：

In [76]:
'''人工提前确定好所有关键字段'''
grxybg_schema = []
grxybg_schema.extend(['报告编号', '报告时间'])
grxybg_schema.extend(['姓名', '证件类型', '证件号码', '是否已婚'])
grxybg_schema.extend(['其他证件信息'])

result = {k:key_2_English(k, force_translate=True) for k in grxybg_schema}
result

{'报告编号': 'report_no',
 '报告时间': 'report_time',
 '姓名': 'name',
 '证件类型': 'certificate_type',
 '证件号码': 'id_number',
 '是否已婚': 'married_or_not',
 '其他证件信息': 'other_document_information'}

## == 提取方案

1. 预处理文本
2. 人工定义所有要提取的关键字段
3. paddleNLP taskflow information extraction 信息抽取
4. 人工调整校验

## == 案例执行结果

------------------------------ 模型抽取 ----------------------------------

In [87]:
""" 加载信息抽取模型 """
## 默认uie_base
grxybg_ie = Taskflow('information_extraction', schema=grxybg_schema)  # 贷记卡逾期

grxybg_result = extract_info_from_taskflow(task=grxybg_ie, case=grxybg_data, schema=grxybg_schema)


grxybg_result

[2024-08-26 17:25:25,921] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'C:\Users\0049004320\.paddlenlp\taskflow\information_extraction\uie-base'.
E:\miniconda3\envs\ZheShang\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:2478: FutureWarning: The `max_seq_len` argument is deprecated and will be removed in a future version, please use `max_length` instead.
  warnings.warn(
E:\miniconda3\envs\ZheShang\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:1878: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15  姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚  其他证件信息：护照G300234234/军人身份证 M09876893


{'报告编号': '2016080303000014210351',
 '报告时间': '2016-08-03 09:30:15',
 '姓名': '张三',
 '证件类型': '身份证',
 '证件号码': '110124197506232452',
 '是否已婚': '已婚',
 '其他证件信息': '护照G300234234'}

- `其他证件信息` 识别不完整

--------------------------------- 人工矫正 -------------------------------

In [22]:
def grxybg_result_fixing(case, result):
    '''
    1. 其他证件信息
        默认该项是最后一个item,直接定位到section最后
    2. 待补充矫正
    '''
    ## 其他证件信息
    item = "其他证件信息"
    matches = list(re.finditer(item, case))
    if len(matches) == 0:
        item_info = None
    else:
        idx = matches[0].end()
        item_info = case[idx:]
        item_info = re.sub(r'[\n:： ]', '', item_info)
    result[item] = item_info

    return result

In [88]:
## 人工矫正
grxybg_result = grxybg_result_fixing(case=grxybg_data, result=grxybg_result)

print(grxybg_data)
## key用英文显示
{key_2_English(k):v for k,v in grxybg_result.items()}


报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15  姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚  其他证件信息：护照G300234234/军人身份证 M09876893


{'Report No.': '2016080303000014210351',
 'Report  Time ': '2016-08-03 09:30:15',
 'Name': '张三',
 'Certificate Type': '身份证',
 'ID Number': '110124197506232452',
 'Married or not': '已婚',
 'Other document information': '护照G300234234/军人身份证M09876893'}

## == 备注

- 提取关键字段信息需业务人员补充
- !! 人工矫正逻辑需测试和业务人员进行矫正+补充

# 信贷记录

## 信息概要

#### == 内容样例

![信息概要](./4_简版方案/信息概要.png)

In [1]:
xxgy_data = []
xxgy_data.append([
    ['', '资产处置信息', '垫款信息'], 
    ['账户数', '1', '3']])

xxgy_data.append([
    ['', '信用卡', '贷款', None, '其他业务'],
    [None, None, '购房', '其他', None],
    ['账户数', '8', '4', '5', '8'],
    [None, None, None, None, None],
    ['未结清/未销户账户数', '4', '2', '3', '7'],
    ['发生过逾期的账户数', '4', '2', '2', '4'],
    ['发生过90天以上逾期的账户数', '4', '1', '1', '1']])

xxgy_data.append([
    ['', '为个人', '为企业'], 
    ['相关还款责任账户数', '1', '3']])

In [65]:
xxgy_data[1]

[['', '信用卡', '贷款', None, '其他业务'],
 [None, None, '购房', '其他', None],
 ['账户数', '8', '4', '5', '8'],
 [None, None, None, None, None],
 ['未结清/未销户账户数', '4', '2', '3', '7'],
 ['发生过逾期的账户数', '4', '2', '2', '4'],
 ['发生过90天以上逾期的账户数', '4', '1', '1', '1']]

#### == 预期提取效果

整体上（提取结果存放在字段 `info_extraction`）：
```json
'header-1-0':{
    'header-name':'信息概要',
    'content':[ocr返回结果],
    'info_extraction':[table_1_info, table_2_info, table_3_info]
}
```
`table_i_info` 样式如下：

一行一行读，嵌套标题会被展开
```json
{
    row_name_1:{
        col_name_1:value, col_name_2:value, ...}
    },
    row_name_2:{
        col_name_1:value, col_name_2:value, ...}
    },
    ...
}
```

#### == 提取方案

1. 预处理
    - 删空值行
    - 默认第一列为行名
    - 无数字的前i(<=2)行为列名
2. 一行一行按规则提取

#### == 案例执行结果

------------------------------ 表格拆解 --------------------------------

In [42]:
def drop_None_row(table:list[list]):
    '''
    drop a row if its all None
    '''
    return [row for row in table if any(e is not None for e in row)]

def is_number(s:str):
    ## 不包含空值和None
    if s is None or s == '':
        return False
    ## 其他值判断是否为纯数字
    try:
        float(s)
        return True
    except ValueError:
        return False
        
def get_col_name_rows(table:list[list]):
    '''
    find first n rows that have no numbers
    '''
    for i in range(len(table)):
        row = table[i]
        flag = not any(is_number(e) for e in row)  ## 是否为标题行
        if flag == False:
            break
    return table[:i]

def is_empty(s:str):
    if s is None or s == '':
        return True
    else:
        return False
        
def combine_col_names(col_name_rows:list[list]):
    '''
    返回：行名、列名、数据
    
    逻辑：
        1. 默认合并单元格的信息在左上格
        2. 从末列向前：
            a. 空值向上传递，遇到非空值被覆盖，继续向上转b
            b. 非空值向上传递，(遇到空值，找该空值左边第一个非空值合并)，继续向上重复
    
    表格转置后可用在合并行名上
    '''
    R = len(col_name_rows)
    C = len(col_name_rows[0])

    col_names = []
    for c in range(C-1, -1, -1):
        col_name = col_name_rows[R-1][c]
        
        empty = False  ## whether col_name is empty now
        if is_empty(col_name):
            empty = True
            
        for r in range(R-2, -1, -1):
            up = col_name_rows[r][c]
            
            if empty:
                if is_empty(up): ## 空值连续向上
                    continue
                else:
                    col_name = up  ## 空值被非空值覆盖
                    empty = False
            else:
                if is_empty(up):
                    ## 非空值向上遇到空值, 找空值左边第一个非空值作为母标题
                    left = ''
                    for c_temp in range(c-1, -1, -1):
                        left = col_name_rows[r][c_temp]
                        if not is_empty(left):
                            col_name = left + "_" + col_name
                            break
                else:
                    col_name = up + "_" + col_name

        if not is_empty(col_name):  ## 存入非空列名
            col_names.append(col_name)

    return col_names[::-1]
                
                
def xxgy_preprocess_table(table:list[list]):  ## 信息概要
    '''
    忽略首格[0][0]（默认为空）
    默认行名在第一列
    
    可修改复用在标准表格上：
        列名在左 + 行名在上 + 数字数据在中间
    '''
    ## 删全为None的行
    table = drop_None_row(table)
    R = len(table)
    C = len(table[0])
    # print(table)

    ## 找'列名'的行:
    col_name_rows = get_col_name_rows(table)
    num_col_name_rows = len(col_name_rows)  ## 前几行是标题行
    # print(num_col_name_rows)
    
    ## 提取列名
    col_names = combine_col_names(col_name_rows)

    ## 提取行名, 默认第一列
    row_names = []
    for r in range(num_col_name_rows, R):
        row_names.append(table[r][0])

    ## 提取数据:
    ##     table里倒数 len(row_names)行, 和倒数 len(col_names)列
    data = []
    for r in range(R-len(row_names), R):
        data_row = []
        for c in range(C-len(col_names), C):
            data_row.append(table[r][c])
        data.append(data_row)

    return row_names, col_names, data    

In [80]:
table_1_result = xxgy_preprocess_table(xxgy_data[0])
table_1_result

(['账户数'], ['资产处置信息', '垫款信息'], [['1', '3']])

In [44]:
table_2_result = xxgy_preprocess_table(xxgy_data[1])
table_2_result

(['账户数', '未结清/未销户账户数', '发生过逾期的账户数', '发生过90天以上逾期的账户数'],
 ['信用卡', '贷款_购房', '贷款_其他', '其他业务'],
 [['8', '4', '5', '8'],
  ['4', '2', '3', '7'],
  ['4', '2', '2', '4'],
  ['4', '1', '1', '1']])

In [45]:
table_3_result = xxgy_preprocess_table(xxgy_data[2])
table_3_result

(['相关还款责任账户数'], ['为个人', '为企业'], [['1', '3']])

------------------------------------ 最终呈献 -----------------------------------

In [53]:
def xxgy_table_result_to_json(row_names, col_names, data):
    table_dict = {}
    for r in range(len(row_names)):
        r_name = row_names[r]
        table_dict[r_name] = {}
        for c in range(len(col_names)):
            c_name = col_names[c]
            table_dict[r_name][c_name]=data[r][c]
    return table_dict  

In [91]:
table_1_info = xxgy_table_result_to_json(row_names=key_list_2_English(table_1_result[0]),
                                         col_names=key_list_2_English(table_1_result[1]),
                                        data=key_list_2_English(table_1_result[2]))
display(pd.DataFrame(xxgy_data[0]))
table_1_info

,0,1,2
0,,资产处置信息,垫款信息
1,账户数,1,3


{'number_of_accounts': {'asset_disposal_information': '1',
  'advance_information': '3'}}

In [92]:
table_2_info = xxgy_table_result_to_json(row_names=key_list_2_English(table_2_result[0]),
                                         col_names=key_list_2_English(table_2_result[1]),
                                        data=key_list_2_English(table_2_result[2]))
display(pd.DataFrame(xxgy_data[1]))
table_2_info

,0,1,2,3,4
0,,信用卡,贷款,None,其他业务
1,None,None,购房,其他,None
2,账户数,8,4,5,8
3,None,None,None,None,None
4,未结清/未销户账户数,4,2,3,7
5,发生过逾期的账户数,4,2,2,4
6,发生过90天以上逾期的账户数,4,1,1,1


{'number_of_accounts': {'credit_cards': '8',
  'loan__buy_home': '4',
  'loan__other': '5',
  'other_business': '8'},
 'number_of_openuncancelled_accounts': {'credit_cards': '4',
  'loan__buy_home': '2',
  'loan__other': '3',
  'other_business': '7'},
 'number_of_accounts_with_records_of_being_overdue': {'credit_cards': '4',
  'loan__buy_home': '2',
  'loan__other': '2',
  'other_business': '4'},
 'number_of_accounts_with_records_of_being_overdue_for_over_90_days': {'credit_cards': '4',
  'loan__buy_home': '1',
  'loan__other': '1',
  'other_business': '1'}}

In [93]:
table_3_info = xxgy_table_result_to_json(row_names=key_list_2_English(table_3_result[0]),
                                         col_names=key_list_2_English(table_3_result[1]),
                                        data=key_list_2_English(table_3_result[2]))
display(pd.DataFrame(xxgy_data[2]))
table_3_info

,0,1,2
0,,为个人,为企业
1,相关还款责任账户数,1,3


{'number_of_related_repayment_liability_accounts': {'for_individuals': '1',
  'for_business': '3'}}

#### == 备注

- !具体操作需根据ocr返回结果修改
- 翻译矫正

### 资产处置信息
简写 -- 具体思路同 `逾期贷记卡`

#### -- 定义关键字段 

In [25]:
'''关键字段需提前确定'''
zcczxx_schema = ["债权接收日期", "债权接收单位", "债权接收金额"]
zcczxx_schema.extend(["余额截至日期", "余额数目", "最近还款日期"])

{k:key_2_English(k) for k in zcczxx_schema}

{'债权接收日期': 'date_of_claim_receipt',
 '债权接收单位': 'entity_receiving_the_claim',
 '债权接收金额': 'amount_of_claim_received',
 '余额截至日期': 'balance_reference_date',
 '余额数目': 'remaining_balance',
 '最近还款日期': 'last_repayment_date'}

In [24]:
## 翻译修正
# update_trans_dict('债权接收日期', 'date_of_claim_receipt')
# update_trans_dict('债权接收单位', 'entity_receiving_the_claim')
# update_trans_dict('债权接收金额', 'amount_of_claim_received')
# update_trans_dict('余额截至日期', 'balance_reference_date')
# update_trans_dict('余额数目', 'remaining_balance')
# update_trans_dict('最近还款日期', 'last_repayment_date')

#### -- 预处理数据

In [12]:
zcczxx_data = """
\n1.2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至 2014年11月11日，余额为920,000，最近一次\n还款日期为2015年12月11日。\n
"""
zcczxx_data

'\n\n1.2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至 2014年11月11日，余额为920,000，最近一次\n还款日期为2015年12月11日。\n\n'

In [18]:
zcczxx_cases = split_cases_by_idx_from_paragraph(zcczxx_data)
zcczxx_cases

['2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至 2014年11月11日，余额为920,000，最近一次还款日期为2015年12月11日。']

#### -- 信息抽取结果

In [11]:
""" 加载信息抽取模型 """
## 默认uie_base
zcczxx_ie = Taskflow('information_extraction', schema=zcczxx_schema)  

[2024-08-28 09:39:04,746] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'C:\Users\0049004320\.paddlenlp\taskflow\information_extraction\uie-base'.


In [46]:
for case in zcczxx_cases:
    ## 模型提取
    result = extract_info_from_taskflow(task=zcczxx_ie, case=case, schema=zcczxx_schema)
    ## key值转英文
    result = {key_2_English(k):v for k,v in result.items()}
    ## 结果展示
    print("-----------")
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

-----------
2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至 2014年11月11日，余额为920,000，最近一次还款日期为2015年12月11日。


,key,value
0,date_of_claim_receipt,2012年03月12日
1,entity_receiving_the_claim,东方资产管理公司
2,amount_of_claim_received,"11,000,000"
3,balance_reference_date,2014年11月11日
4,remaining_balance,"920,000"
5,last_repayment_date,2015年12月11日


### 垫款信息 
简写 -- 具体思路同 `逾期贷记卡`

#### -- 定义关键字段

In [30]:
'''关键字段需提前确定'''
dkxx_schema = ["累计代偿金额开始日期", "累计代偿金额所属单位", "累计代偿金额数目"]
dkxx_schema.extend(["是否已结清", "结清日期"])
dkxx_schema.extend(["余额截至日期", "余额数目", "最近还款日期"])

{k:key_2_English(k) for k in dkxx_schema}

{'累计代偿金额开始日期': 'cumulative_compensation_amount_start_date',
 '累计代偿金额所属单位': 'entity_for_cumulative_compensation_amount',
 '累计代偿金额数目': 'cumulative_compensation_amount',
 '是否已结清': 'is_settled',
 '结清日期': 'settlement_date',
 '余额截至日期': 'balance_reference_date',
 '余额数目': 'remaining_balance',
 '最近还款日期': 'last_repayment_date'}

In [29]:
## 翻译修正
# update_trans_dict("累计代偿金额所属单位", "entity_for_cumulative_compensation_amount")
# update_trans_dict("是否已结清", "is_settled")

#### -- 预处理数据

In [31]:
dkxx_data = """
\n1.2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为\n2014年10月10日。\n2.2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。\n
"""
dkxx_data

'\n\n1.2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为\n2014年10月10日。\n2.2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。\n\n'

In [32]:
dkxx_cases = split_cases_by_idx_from_paragraph(dkxx_data)
dkxx_cases

['2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为2014年10月10日。',
 '2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。']

#### -- 信息抽取结果

In [34]:
""" 加载信息抽取模型 """
## 默认uie_base
dkxx_ie = Taskflow('information_extraction', schema=dkxx_schema)  

[2024-08-28 10:47:16,562] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'C:\Users\0049004320\.paddlenlp\taskflow\information_extraction\uie-base'.


In [ ]:
""" 人工矫正抽取信息 """
def dkxx_result_fixing(case, result):
    '''
    矫正逻辑：
        1. 若含有“已结清”：
            是否已结清：是
            余额截至日期：None
            余额数目: None
        2. 若不含有“已结清”：
            是否已结清：否
            结清日期：None
    '''
    case = case.replace(" ", "")  ## 删空格
    
    if '已结清' in case:
        result["是否已结清"] = '是'
        result["余额截至日期"] = None
        result["余额截至日期"] = None
        result["余额数目"] = None
    else:
        result["是否已结清"] = '否'
        result["结清日期"] = None

    return result


In [108]:
for case in dkxx_cases:
    ## 模型提取
    result = extract_info_from_taskflow(task=dkxx_ie, case=case, schema=dkxx_schema)
    ## 结果矫正
    result = dkxx_result_fixing(case, result)
    ## key值转英文
    result = {key_2_English(k):v for k,v in result.items()}
    ## 结果展示
    print("-----------")
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

-----------
2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为2014年10月10日。


,key,value
0,cumulative_compensation_amount_start_date,2012年02月02日
1,entity_for_cumulative_compensation_amount,中国人寿保险公司
2,cumulative_compensation_amount,80，000
3,is_settled,否
4,settlement_date,None
5,balance_reference_date,2014年09月9日
6,remaining_balance,"70,000"
7,last_repayment_date,2014年10月10日


-----------
2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。


,key,value
0,cumulative_compensation_amount_start_date,2013年02月02日
1,entity_for_cumulative_compensation_amount,富登融资租赁担保公司
2,cumulative_compensation_amount,"400,000"
3,is_settled,是
4,settlement_date,2016年1月
5,balance_reference_date,None
6,remaining_balance,None
7,last_repayment_date,None


#### -- 备注

- !! 矫正规则需业务人员修改完善

## 信用卡

### 发生过逾期的贷记卡账户明细如下：

#### == 内容样例

![发生过逾期的贷记卡账户明细](./4_简版方案/发生过逾期的贷记卡账户明细.png)

In [9]:
djkyq_data = """
\n1. 2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度\n10,000，已变成呆账，余额500。\n2. 2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，\n余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超\n过90天。\n3. 2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内\n有7个月处于逾期状态，其中3个月逾期超过90天。\n2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。\n
"""
print(djkyq_data)



1. 2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度
10,000，已变成呆账，余额500。
2. 2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，
余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超
过90天。
3. 2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内
有7个月处于逾期状态，其中3个月逾期超过90天。
2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。




In [17]:
"""预处理分割data"""
cases = split_cases_by_idx_from_paragraph(data=djkyq_data)
case_1, case_2, case_3 = cases[0], cases[1], cases[2]
print(f"case_1: {case_1} \n")
print(f"case_2: {case_2} \n")
print(f"case_3: {case_3} \n")

case_1: 2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度10,000，已变成呆账，余额500。 

case_2: 2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。 

case_3: 2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。 



#### == 预期提取效果

整体（提取结果存放在字段 `info_extraction`）：
```json
{
    'header-1-1-0':{
        'header-name':'发生过逾期的贷记卡账户明细',
        'content': [ocr返回结果]
        'info_extraction':[case_0_info, case_1_info, case_2_info]
    }
}
```
`case_i_info` 关键字段及其中英对照：

In [96]:
'''人工提前定好schema内容'''
djkyq_schema = ['发卡日期', '发卡机构','账户币种','卡片尾号','销户日期']
djkyq_schema.extend(['信用额度截至日期','信用额度','是否变成呆账','呆账余额','当前有无逾期'])
djkyq_schema.extend(['近5年处于逾期状态月数','逾期超过90天月数'])  ## ?? '5年'定死、?? '90天'定死

case_i_info = {k:key_2_English(k) for k in djkyq_schema}
case_i_info

{'发卡日期': 'card_issue_date',
 '发卡机构': 'card_issuer',
 '账户币种': 'account_currency',
 '卡片尾号': 'card_suffix',
 '销户日期': 'account_canceling_date',
 '信用额度截至日期': 'reference_date',
 '信用额度': 'credit_limit',
 '是否变成呆账': 'whether_it_turns_into_bad_debts',
 '呆账余额': 'bad_debt_balance',
 '当前有无逾期': 'is_currently_overdue',
 '近5年处于逾期状态月数': 'number_of_months_overdue_in_the_last_5_years',
 '逾期超过90天月数': 'number_of_months_overdue_for_more_than_90_days'}

In [95]:
"""手动矫正翻译"""
# update_trans_dict("销户日期", "account_canceling_date")
# update_trans_dict("卡片尾号", "card_suffix")
# update_trans_dict("信用额度截至日期", "reference_date")
# update_trans_dict("近5年处于逾期状态月数", "number_of_months_overdue_in_the_last_5_years")
# update_trans_dict("逾期超过90天月数", "number_of_months_overdue_for_more_than_90_days")
# update_trans_dict("发卡机构", "card_issuer")
# update_trans_dict("发卡日期", "card_issue_date")

#### == 提取方案

1. 文本预处理，手动分割段落
1. 人工定义提取关键字段 `djkyq_schema`
   - 待补充 （请业务人员给出建议）
2. paddleNLP taskflow information extraction 用小模型进行字段信息抽取
3. 后续人工用规则矫正
    - 详见下文 `人工修正函数`
    - !! 待补充（请测试/业务人员给出建议）
    

#### == 案例执行结果

------------------------------ 模型抽取 ----------------------------------

In [107]:
""" 加载信息抽取模型 """
## 默认uie_base
djkyq_ie = Taskflow('information_extraction', schema=djkyq_schema)  # 贷记卡逾期

[2024-08-27 19:12:41,669] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'C:\Users\0049004320\.paddlenlp\taskflow\information_extraction\uie-base'.


In [105]:
print(case_1)
result_1 = extract_info_from_taskflow(task=djkyq_ie, case=case_1, schema=djkyq_schema)
result_1

2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度10,000，已变成呆账，余额500。


{'发卡日期': '2004 年 8 月 30 日',
 '发卡机构': '中国建设银行北京分行',
 '账户币种': '人民币',
 '卡片尾号': '0001',
 '销户日期': None,
 '信用额度截至日期': '2010 年 10 月',
 '信用额度': '10,000',
 '是否变成呆账': None,
 '呆账余额': '500',
 '当前有无逾期': None,
 '近5年处于逾期状态月数': None,
 '逾期超过90天月数': None}

- 无法识别`是否变成呆账`

In [101]:
print(case_2)
result_2 = extract_info_from_taskflow(task=djkyq_ie, case=case_2, schema=djkyq_schema)
result_2

2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。


{'发卡日期': '2004年8月30日',
 '发卡机构': '中国工商银行北京分行',
 '账户币种': '人民币',
 '卡片尾号': '0002',
 '销户日期': None,
 '信用额度截至日期': '2016年7月',
 '信用额度': '50,000',
 '是否变成呆账': None,
 '呆账余额': '5,000',
 '当前有无逾期': None,
 '近5年处于逾期状态月数': '11个月',
 '逾期超过90天月数': None}

In [106]:
case_2_1 = "2.1. 2004 年 8 月 30 日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至 2016 年 7 月，信用额度 50,000，余额 5,000（含未出单的大额专项分期余额 4,000），当前有逾期。最近 6 年内有 11 个月处于逾期状态，其中 5 个月逾期超过 80 天。"
print(case_2_1)
result_2_1 = extract_info_from_taskflow(task=djkyq_ie, case=case_2_1, schema=djkyq_schema)
result_2_1

2.1. 2004 年 8 月 30 日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至 2016 年 7 月，信用额度 50,000，余额 5,000（含未出单的大额专项分期余额 4,000），当前有逾期。最近 6 年内有 11 个月处于逾期状态，其中 5 个月逾期超过 80 天。


{'发卡日期': '2004 年 8 月 30 日',
 '发卡机构': '中国工商银行北京分行',
 '账户币种': '人民币',
 '卡片尾号': '0002',
 '销户日期': None,
 '信用额度截至日期': '2016 年 7 月',
 '信用额度': '50,000',
 '是否变成呆账': None,
 '呆账余额': '5,000',
 '当前有无逾期': None,
 '近5年处于逾期状态月数': '11 个月',
 '逾期超过90天月数': '5 个月'}

- 识别错误（数据内改 `5年` 为 `6年`，改 `90天` 为 `80天`）

In [102]:
print(case_3)
result_3 = extract_info_from_taskflow(task=djkyq_ie, case=case_3, schema=djkyq_schema)
result_3

2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。


{'发卡日期': '2010年4月1日',
 '发卡机构': '中国民生银行信用卡中心',
 '账户币种': '人民币',
 '卡片尾号': '0003',
 '销户日期': '2015年12月',
 '信用额度截至日期': None,
 '信用额度': None,
 '是否变成呆账': None,
 '呆账余额': None,
 '当前有无逾期': None,
 '近5年处于逾期状态月数': '7个月',
 '逾期超过90天月数': '3个月'}

--------------------------------- 人工矫正 -------------------------------

In [103]:
""" 定义人工修正函数 """
def djkyq_result_fixing(case, result):
    '''
    修正逻辑：
        1. 无法识别`'是否变成呆账'`:
            若含有 `呆账`, 则改对应值为 `是`
        2. 逾期状态识别不一定精确识别原文数字改动：
            若不含有`近5年内`, 则改`'近5年处于逾期状态月数'`对应值为`None`
            若不含有`超过90天`, 则改`'逾期超过90天月数'`对应值为`None`
        3. ...
    '''
    case = case.replace(" ", "")  ## 删空格
    
    if '呆账' in case:
        result["是否变成呆账"] = '是'

    if '近5年内' not in case:
        result["近5年处于逾期状态月数"] = None
    if '超过90天' not in case:
        result["逾期超过90天月数"] = None
        
    return result

In [108]:
## 矫正
result_1 = djkyq_result_fixing(case=case_1, result=result_1)

result_2 = djkyq_result_fixing(case=case_2, result=result_2)

result_2_1 = djkyq_result_fixing(case=case_2_1, result=result_2_1)

result_3 = djkyq_result_fixing(case=case_3, result=result_3)

------------------------------ 最终效果展示 -----------------------------

In [109]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

cases = [case_1, case_2, case_2_1, case_3]
results = [result_1, result_2, result_2_1, result_3]
for i in range(len(cases)):
    print("--------------")
    print(cases[i])
    result = {key_2_English(k):v for k,v in results[i].items()}
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

--------------
2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度10,000，已变成呆账，余额500。


,key,value
0,card_issue_date,2004 年 8 月 30 日
1,card_issuer,中国建设银行北京分行
2,account_currency,人民币
3,card_suffix,0001
4,account_canceling_date,None
5,reference_date,2010 年 10 月
6,credit_limit,"10,000"
7,whether_it_turns_into_bad_debts,是
8,bad_debt_balance,500
9,is_currently_overdue,None


--------------
2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。


,key,value
0,card_issue_date,2004年8月30日
1,card_issuer,中国工商银行北京分行
2,account_currency,人民币
3,card_suffix,0002
4,account_canceling_date,None
5,reference_date,2016年7月
6,credit_limit,"50,000"
7,whether_it_turns_into_bad_debts,None
8,bad_debt_balance,"5,000"
9,is_currently_overdue,None


--------------
2.1. 2004 年 8 月 30 日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至 2016 年 7 月，信用额度 50,000，余额 5,000（含未出单的大额专项分期余额 4,000），当前有逾期。最近 6 年内有 11 个月处于逾期状态，其中 5 个月逾期超过 80 天。


,key,value
0,card_issue_date,2004 年 8 月 30 日
1,card_issuer,中国工商银行北京分行
2,account_currency,人民币
3,card_suffix,0002
4,account_canceling_date,None
5,reference_date,2016 年 7 月
6,credit_limit,"50,000"
7,whether_it_turns_into_bad_debts,None
8,bad_debt_balance,"5,000"
9,is_currently_overdue,None


--------------
2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。


,key,value
0,card_issue_date,2010年4月1日
1,card_issuer,中国民生银行信用卡中心
2,account_currency,人民币
3,card_suffix,0003
4,account_canceling_date,2015年12月
5,reference_date,None
6,credit_limit,None
7,whether_it_turns_into_bad_debts,None
8,bad_debt_balance,None
9,is_currently_overdue,None


#### == 备注

1. 已忽略最后一行 `2014 年 3 月，该机构声明：该客户委托 XX 公司偿还贷款，因开发公司不按时还款导致出现多次逾期。`
   - （处理逻辑是啥，会有别的类似内容吗）
2. 已忽略 `case_2` 中 `（含未出单的大额专项分期余额 4,000）`
   - （处理逻辑是啥，会有别的括号内容吗）
3. !!! `确定关键字段` 和 `手工矫正` 部分需要专业人员进行补充改正
4. 英文翻译需业务人员矫正
5. 操作细节需根据ocr返回结果调整

### 透支超过 60 天的准贷记卡账户明细如下：
略

### 从未逾期过的贷记卡及透支未超过 60 天的准贷记卡账户明细如下：
略

## 贷款

### 发生过逾期的账户明细如下：
略

## 其他业务

### 发生过逾期的账户明细如下：
略

In [104]:
qtyw_fsgyq_schema = ["业务开始日期", "业务办理单位", "业务办理金额", "业务结束日期"]
qtyw_fsgyq_schema.extend(["截至日期","是否为呆账", "最近还款日期", "余额数目"])
qtyw_fsgyq_schema.extend(['近5年处于逾期状态月数','逾期超过90天月数'])

{k:key_2_English(k) for k in qtyw_fsgyq_schema}

{'业务开始日期': 'start_date_of_business',
 '业务办理单位': 'business_handling_unit',
 '业务办理金额': 'business_transaction_amount',
 '业务结束日期': 'business_end_date',
 '截至日期': 'reference_date',
 '是否为呆账': 'whether_it_is_a_bad_debt',
 '最近还款日期': 'last_repayment_date',
 '余额数目': 'remaining_balance',
 '近5年处于逾期状态月数': 'number_of_months_overdue_in_the_last_5_years',
 '逾期超过90天月数': 'number_of_months_overdue_for_more_than_90_days'}

In [105]:
qtyw_fsgyq_data = """
\n1. 2005年10月20日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年10月20日到期。\n截至2011年11月，已变成呆账，最近一次还款日期为2010年11月25日，余额150,000。\n2. 2005年9月22日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年9月22日到期。\n截至2016年7月，余额400,000，当前无逾期。最近5年内有3个月处于逾期状态，没有发生过90天以上逾期。\n3. 2006年10月20日中信证券公司办理的500,000元（人民币）约定购回式证券交易，2007年10月19日到期。截至2008年1\n月，已变成司法追偿，最近一次还款日期为2007年10月15日，余额150,000。\n4. 2009年9月22日海通证券公司办理的500,000元（人民币）约定购回式证券交易，2010年9月21日到期。截至2016年7\n月，余额500,000，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。\n
"""
qtyw_fsgyq_data

'\n\n1. 2005年10月20日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年10月20日到期。\n截至2011年11月，已变成呆账，最近一次还款日期为2010年11月25日，余额150,000。\n2. 2005年9月22日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年9月22日到期。\n截至2016年7月，余额400,000，当前无逾期。最近5年内有3个月处于逾期状态，没有发生过90天以上逾期。\n3. 2006年10月20日中信证券公司办理的500,000元（人民币）约定购回式证券交易，2007年10月19日到期。截至2008年1\n月，已变成司法追偿，最近一次还款日期为2007年10月15日，余额150,000。\n4. 2009年9月22日海通证券公司办理的500,000元（人民币）约定购回式证券交易，2010年9月21日到期。截至2016年7\n月，余额500,000，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。\n\n'

In [106]:
qtyw_fsgyq_cases = split_cases_by_idx_from_paragraph(qtyw_fsgyq_data)
qtyw_fsgyq_cases

['2005年10月20日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年10月20日到期。截至2011年11月，已变成呆账，最近一次还款日期为2010年11月25日，余额150,000。',
 '2005年9月22日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年9月22日到期。截至2016年7月，余额400,000，当前无逾期。最近5年内有3个月处于逾期状态，没有发生过90天以上逾期。',
 '2006年10月20日中信证券公司办理的500,000元（人民币）约定购回式证券交易，2007年10月19日到期。截至2008年1月，已变成司法追偿，最近一次还款日期为2007年10月15日，余额150,000。',
 '2009年9月22日海通证券公司办理的500,000元（人民币）约定购回式证券交易，2010年9月21日到期。截至2016年7月，余额500,000，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。']

In [ ]:
""" 加载信息抽取模型 """
## 默认uie_base
djkyq_ie = Taskflow('information_extraction', schema=djkyq_schema) 

### 从未逾期过的账户明细如下：
略

In [ ]:
qtyw_cwyq_schema = ["办理业务日期", "业务办理单位", "办理业务名称", ""

## 相关还款责任信息
略

# 非信贷交易记录

```json
{
        'header-2':{
        'header-name':'非信贷交易记录',
        'content':'这部分包含您最近 5 年内的非信贷交易记录。金额类数据均以人民币计算，精确到元。'
        'section':{
            'header-2-0':{...},
            'header-2-1':{...},
            ...
        }
    },
}
```

## 后付费记录

### == 内容样例

![后付费记录](4_简版方案/后付费记录.png)

In [53]:
hffjl_data = []
hffjl_data.append(['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月'])
hffjl_data.append(['业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550'])
hffjl_data.append(['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月'])
hffjl_data.append(['业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0'])
hffjl_data.append(['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月'])
hffjl_data.append(['业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0'])
hffjl_data

[['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月'],
 ['业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550'],
 ['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月'],
 ['业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0'],
 ['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月'],
 ['业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0']]

In [54]:
""" 预处理分割data """
def hffjl_split_cases(data):  ## 后付费记录
    cases = []
    n_line_split = 2  ## 固定2行一组
    for i in range(len(data)):
        if i % n_line_split == 0:
            cases.append(data[i])
        else:
            cases[-1].extend(data[i])
    return cases

hffjl_cases = hffjl_split_cases(data=hffjl_data)
hffjl_cases

[['机构名称：中国电信北京分公司',
  '业务类型：固定电话后付费',
  '记账年月：2016年7月',
  '业务开通日期：2012年6月28日',
  '当前缴费状态：欠费',
  '当前欠费金额：550'],
 ['机构名称：中国移动北京分公司',
  '业务类型：移动电话后付费',
  '记账年月：2016年6月',
  '业务开通日期：2013年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0'],
 ['机构名称：北京自来水公司',
  '业务类型：自来水费',
  '记账年月：2016年6月',
  '业务开通日期：2014年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0']]

### == 预期提取效果

整体（提取结果存放在字段 `info_extraction`）：
```json
{
    'header-2-0':{
        'header-name':'后付费记录',
        'content':[ocr返回结果],
        'info_extraction':[case_0_info, case_1_info, case_2_info],
    }
}
```
`case_i_info` 关键字段及其翻译如下：

In [52]:
'''
此处schema应由规则提取得到，
而非人为规定
'''
hffjl_schema = ["机构名称", "业务类型", "记账年月"]
hffjl_schema.extend(["业务开通日期", "当前缴费状态", "当前欠费金额"])

case_i_info = {k:key_2_English(k) for k in hffjl_schema}
case_i_info

{'机构名称': 'organization_name',
 '业务类型': 'business_type',
 '记账年月': 'accounting_year_month',
 '业务开通日期': 'business_activation_date',
 '当前缴费状态': 'current_billing_status',
 '当前欠费金额': 'current_arrears'}

### == 提取方案

- 手动规则提取
    - 先识别出一个一个案例（e.g. 固定两行为一组）
    - 键值对抽取：" "/": "/":"进行分割, 前面为key, 后面为value

### == 案例执行结果

In [55]:
def hffjl_extract_info_from_caselist(cases):
    results = []
    for case in cases:
        result = {}
        for k_v in case:
            if k_v =='':
                continue
            k_v = re.sub(r'[:：]', '|', k_v)  ## 冒号分割
            k_v = re.split(r'\|+', k_v)
            k = re.sub(r'\s+', '', k_v[0])  ## 去除空白
            v = re.sub(r'\s+', '', k_v[1])
            result[k] = v
        results.append(result)
    return results

hffjl_results = hffjl_extract_info_from_caselist(hffjl_cases)

In [56]:
for i in range (len(hffjl_cases)):
    print("--------------")
    print(hffjl_cases[i])
    result = {key_2_English(k):v for k,v in hffjl_results[i].items()}
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

--------------
['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月', '业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550']


,key,value
0,organization_name,中国电信北京分公司
1,business_type,固定电话后付费
2,accounting_year_month,2016年7月
3,business_activation_date,2012年6月28日
4,current_billing_status,欠费
5,current_arrears,550


--------------
['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月', '业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0']


,key,value
0,organization_name,中国移动北京分公司
1,business_type,移动电话后付费
2,accounting_year_month,2016年6月
3,business_activation_date,2013年1月31日
4,current_billing_status,正常
5,current_arrears,0


--------------
['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月', '业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0']


,key,value
0,organization_name,北京自来水公司
1,business_type,自来水费
2,accounting_year_month,2016年6月
3,business_activation_date,2014年1月31日
4,current_billing_status,正常
5,current_arrears,0


### == 备注

- 具体操作需根据ocr返回结果格式调整

# 公共记录

## 欠税记录
略 -- 方案同 `后付费记录`

- 固定两行一组
- 预期识别关键字段名称:

In [59]:
qsjl_schema = ['主管税务机关', '欠税统计日期', '欠税总额', '纳税人识别号']
{k:key_2_English(k) for k in qsjl_schema}

{'主管税务机关': 'tax_authorities',
 '欠税统计日期': 'tax_arrears_statistics_date',
 '欠税总额': 'total_tax_arrears',
 '纳税人识别号': 'taxpayer_id'}

## 民事判决记录
略 -- 方案同 `后付费记录`

- 定死5行一组
- 预期识别关键字段名称：

In [63]:
mspjjl_schema = ['立案法院', '案号', '案由', '结案方式', '立案日期', '判决/调解结果', '诉讼标的', '判决/调解生效日期', '诉讼标的金额']
{k:key_2_English(k) for k in mspjjl_schema}

{'立案法院': 'filing_court',
 '案号': 'case_number',
 '案由': 'case_reason',
 '结案方式': 'case_closure_method',
 '立案日期': 'filing_date',
 '判决/调解结果': 'judgment_or_mediation_result',
 '诉讼标的': 'litigation_subject',
 '判决/调解生效日期': 'judgment_or_mediation_effective_date',
 '诉讼标的金额': 'litigation_amount'}

## 强制执行记录
略 -- 方案同 `后付费记录`

- 固定6行一组
- 预期识别关键字段名称：

In [66]:
qzzxjl_schema = ['执行法院', '案号', '执行案由', '结案方式', '立案日期', '案件状态', '申请执行标的', '已执行标的', '申请执行标的金额', '已执行标的金额', '结案日期']

{k:key_2_English(k) for k in qzzxjl_schema}

{'执行法院': 'executing_court',
 '案号': 'case_number',
 '执行案由': 'reason_for_execution',
 '结案方式': 'case_closure_method',
 '立案日期': 'filing_date',
 '案件状态': 'case_status',
 '申请执行标的': 'claimed_execution_subject',
 '已执行标的': 'executed_subject',
 '申请执行标的金额': 'claimed_execution_amount',
 '已执行标的金额': 'executed_amount',
 '结案日期': 'closure_date'}

## 行政处罚记录
略 -- 方案同 `后付费记录`

- 固定4行一组
- 预期识别关键字段名称：

In [70]:
xzcfjl_schema = ['处罚机构', '文书编号', '处罚内容', '行政复议结果', '处罚金额', '生效日期', '截止日期']

{k:key_2_English(k) for k in xzcfjl_schema}

{'处罚机构': 'penalty_authority',
 '文书编号': 'document_number',
 '处罚内容': 'penalty_details',
 '行政复议结果': 'administrative_review_result',
 '处罚金额': 'penalty_amount',
 '生效日期': 'effective_date',
 '截止日期': 'due_date'}

# 查询记录

In [93]:
def cxjl_extract_info_from_table(table:list[list], need_translate=True):
    '''
    适用于 第一行为列名, 无行名 的表格
    '''
    col_names = table[0]
    if need_translate:
        col_names = key_list_2_English(col_names)

    results = []
    for r in range(1, len(table)):
        result = {}
        for c in range(len(col_names)):
            result[col_names[c]] = table[r][c]
        results.append(result)

    return results

## 机构查询记录明细 

### == 内容样例

![机构查询记录](4_简版方案/机构查询记录.png)

In [71]:
jgcsjl_data = [
    ['编号', '查询日期', '查询机构', '查询原因'],
    ['1', '2016年7月15日', '中国工商银行', '贷后管理'],
    ['2', '2015年12月10日', '中国农业银行', '贷款审批'],
    ['3', '2015年2月12日', '中国农业银行', '担保资格审查'],
    ['4', '2015年2月10日', '建元资本', '融资审批'],
    ['5', '2015年2月1日', '平安财险', '保前审查']
]

### == 预期提取效果

整体（提取结果存放在字段 `info_extraction`）：
```json
{
    'header-4-0':{
        'header-name':'机构查询记录明细',
        'content':[ocr返回结果],
        'info_extraction':[case_0_info, case_1_info, ..., case_4_info],
    }
}
```
`case_i_info` 关键字段及其翻译如下：

In [100]:
'''
此处schema应由规则提取得到，
而非人为规定
'''
csjl_schema = ['编号', '查询日期', '查询机构', '查询原因']

case_i_info = {k:key_2_English(k) for k in csjl_schema}
case_i_info

{'编号': 'item_no',
 '查询日期': 'inquiry_date',
 '查询机构': 'inquiry_entity',
 '查询原因': 'inquiry_reason'}

### == 提取方案

- 默认第一行为列名
- 默认第二行开始每一行为一份数据

### == 案例执行效果

In [97]:
jgcsjl_info_extraction = cxjl_extract_info_from_table(jgcsjl_data)
jgcsjl_info_extraction

[{'item_no': '1',
  'inquiry_date': '2016年7月15日',
  'inquiry_entity': '中国工商银行',
  'inquiry_reason': '贷后管理'},
 {'item_no': '2',
  'inquiry_date': '2015年12月10日',
  'inquiry_entity': '中国农业银行',
  'inquiry_reason': '贷款审批'},
 {'item_no': '3',
  'inquiry_date': '2015年2月12日',
  'inquiry_entity': '中国农业银行',
  'inquiry_reason': '担保资格审查'},
 {'item_no': '4',
  'inquiry_date': '2015年2月10日',
  'inquiry_entity': '建元资本',
  'inquiry_reason': '融资审批'},
 {'item_no': '5',
  'inquiry_date': '2015年2月1日',
  'inquiry_entity': '平安财险',
  'inquiry_reason': '保前审查'}]

### == 备注

- 具体操作需根据ocr返回结果格式调整

## 本人查询记录明细
略 -- 方案同`机构查新记录明细`

- 默认第一行为列名，无行名
- 预期提取字段：

In [101]:
csjl_schema = ['编号', '查询日期', '查询机构', '查询原因']

{k:key_2_English(k) for k in csjl_schema}

{'编号': 'item_no',
 '查询日期': 'inquiry_date',
 '查询机构': 'inquiry_entity',
 '查询原因': 'inquiry_reason'}